In [1]:
import pandas as pd
import numpy as np

## Initialization

Create the base network as a matrix of costs, where a cost of NaN represents a non-adjacent node. The cost to the same node (diagonal of the matrix) is also listed as NaN for programming purposes. Typically, the diagonal should be listed as a 0, but this creates problems in the scripts for Djikstra's Algorithm and the Bellman-Ford Algorithm.

In [2]:
# Manually initialized adjacency matrix
network = pd.DataFrame(
    [[0,5,1,np.nan,np.nan,np.nan],
     [np.nan,0,4,6,np.nan,np.nan],
     [np.nan,np.nan,0,7,2,np.nan],
     [np.nan,np.nan,np.nan,0,np.nan,2],
     [np.nan,1,np.nan,3,0,8],
     [np.nan,np.nan,np.nan,np.nan,np.nan,0]])

network.index = ['Node 1', 'Node 2', 'Node 3', 'Node 4', 'Node 5', 'Node 6']
network.columns = network.index

display(network)

,Node 1,Node 2,Node 3,Node 4,Node 5,Node 6
Node 1,0.0,5.0,1.0,NaN,NaN,NaN
Node 2,NaN,0.0,4.0,6.0,NaN,NaN
Node 3,NaN,NaN,0.0,7.0,2.0,NaN
Node 4,NaN,NaN,NaN,0.0,NaN,2.0
Node 5,NaN,1.0,NaN,3.0,0.0,8.0
Node 6,NaN,NaN,NaN,NaN,NaN,0.0


## Djikstra's Algorithm

In [3]:
# Initialize the temporary labels, permanent labels, and optimal policy vectors
permanent = pd.Series([0,np.nan,np.nan,np.nan,np.nan,np.nan])
temp = pd.Series([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])

temp.index = permanent.index = network.index

In [4]:
previousPermanentNodeIndex = 0
iteration = 1

while permanent.isnull().values.any():
    
    # Print iteration number
    print(f'Iteration {iteration} permanent costs')

    # Create copy of new possible node costs and set the previous
    # permanent node cost to NaN to avoid an infinite loop caused by the 0 value attached
    # to the same node
    newPossibleNodeCosts = network.iloc[previousPermanentNodeIndex].copy()
    newPossibleNodeCosts.iloc[previousPermanentNodeIndex] = np.nan

    # Calculate the new possible node costs using the previous permanent node
    newNodeCosts = newPossibleNodeCosts + permanent.iloc[previousPermanentNodeIndex]

    # Find min between new possible node costs and temporary costs, call it calcCosts
    calcCosts = np.fmin(newNodeCosts, temp)

    # Find min of the calcCosts and the associated node, make the node
    # and the cost permanent
    newMin = np.nanmin(calcCosts)
    newMinIndex = np.argmin(calcCosts)

    # Update the permanent costs vector
    permanent.iloc[newMinIndex] = newMin
    print(permanent)

    # Update the temporary costs vector
    temp = np.fmin(calcCosts, temp)
    # Make sure that any permanent nodes aren't included
    temp.loc[permanent.notna()] = np.nan


    # Update the previousPermanentNodeIndex
    previousPermanentNodeIndex = newMinIndex

    # Update iteration for display purposes
    iteration += 1

Iteration 1 permanent costs
Node 1    0.0
Node 2    NaN
Node 3    1.0
Node 4    NaN
Node 5    NaN
Node 6    NaN
dtype: float64
Iteration 2 permanent costs
Node 1    0.0
Node 2    NaN
Node 3    1.0
Node 4    NaN
Node 5    3.0
Node 6    NaN
dtype: float64
Iteration 3 permanent costs
Node 1    0.0
Node 2    4.0
Node 3    1.0
Node 4    NaN
Node 5    3.0
Node 6    NaN
dtype: float64
Iteration 4 permanent costs
Node 1    0.0
Node 2    4.0
Node 3    1.0
Node 4    6.0
Node 5    3.0
Node 6    NaN
dtype: float64
Iteration 5 permanent costs
Node 1    0.0
Node 2    4.0
Node 3    1.0
Node 4    6.0
Node 5    3.0
Node 6    8.0
dtype: float64


### Finding optimal path policy

In [5]:
def findPredecessorsDjikstra(targetNode, network, optimalCosts):
    # Iterate through the nodes that are adjacent to the target node unless it's the first node
    if targetNode == 'Node 1':
        return(('Node 1',))

    # Find nodes that are adjacent to the target node and set the cost of same node to NaN so
    # that it isn't considered
    connectingCosts = network.loc[:,targetNode].copy()
    connectingCosts.loc[targetNode] = np.nan

    # Iterate through adjacent nodes
    for adjacentNode in connectingCosts.index:
        
        # If the node is not adjacent, ignore it
        if pd.isna(connectingCosts[adjacentNode]):
            continue
        
        # Check if the cost to get to an adjacent node + the cost to get to target node is equal or
        # less than the optimal cost--if so, the adjacent node is a predecessor
        if optimalCosts[adjacentNode] + connectingCosts[adjacentNode] == optimalCosts[targetNode]:
            return findPredecessorsDjikstra(adjacentNode, network, optimalCosts) + (targetNode,)

In [6]:
# Save a Series with the optimal policy
policyDjikstra = pd.Series(index=network.columns, dtype=object)

for node in network.columns:
    policyDjikstra[node] = findPredecessorsDjikstra(node, network, permanent)

display(policyDjikstra)

Node 1                                   (Node 1,)
Node 2            (Node 1, Node 3, Node 5, Node 2)
Node 3                            (Node 1, Node 3)
Node 4            (Node 1, Node 3, Node 5, Node 4)
Node 5                    (Node 1, Node 3, Node 5)
Node 6    (Node 1, Node 3, Node 5, Node 4, Node 6)
dtype: object

## Bellman-Ford Algorithm

In [7]:
# Initialize costs, policy vectors, and predecessors
nodeCosts = pd.Series([0, np.nan, np.nan, np.nan, np.nan, np.nan], index=network.index)
previousCosts = pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan], index=network.index)
predecessors = pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan], index=network.index, dtype=object)

# Iterate the amount of nodes in the network
# While Bellman-Ford only needs V - 1 iterations, we run V to check for negative cycles
for i in range(len(nodeCosts)):
    
    # Check for negative cycle, detected if i reaches the Vth iteration
    if i == len(nodeCosts):
        print("There was a negative cycle in the network")
    else:
        print(f"Iteration {i+1}")

    # For each iteration, iterate through all starting nodes to see if a smaller
    # cost can be found to a connecting/adjacent end node
    for startNodeIndex in nodeCosts.index:
        for endNodeIndex in nodeCosts.index:
            if startNodeIndex != endNodeIndex:
                checkCost = network.loc[startNodeIndex][endNodeIndex] + nodeCosts[startNodeIndex]
                # If a smaller cost is found, update the cost and the predecessor
                if np.isnan(nodeCosts[endNodeIndex]) or checkCost < nodeCosts[endNodeIndex]:
                    nodeCosts[endNodeIndex] = checkCost
                    predecessors[endNodeIndex] = startNodeIndex
    print(nodeCosts)

    # Check if nodeCosts was updated at all or not
    # If not, the nodeCosts have converged and we can break the loop
    if pd.Series.equals(nodeCosts, previousCosts):
        print("The node costs have converged")
        break

    # Update the previous costs and policy vector for the next iteration
    previousCosts = nodeCosts.copy()

print("\nNode predecessors:")
print(predecessors)

Iteration 1
Node 1     0.0
Node 2     4.0
Node 3     1.0
Node 4     6.0
Node 5     3.0
Node 6    10.0
dtype: float64
Iteration 2
Node 1    0.0
Node 2    4.0
Node 3    1.0
Node 4    6.0
Node 5    3.0
Node 6    8.0
dtype: float64
Iteration 3
Node 1    0.0
Node 2    4.0
Node 3    1.0
Node 4    6.0
Node 5    3.0
Node 6    8.0
dtype: float64
The node costs have converged

Node predecessors:
Node 1       NaN
Node 2    Node 5
Node 3    Node 1
Node 4    Node 5
Node 5    Node 3
Node 6    Node 4
dtype: object


### Finding optimal path policy

In [8]:
# Save a Series with the optimal policy using the predecessors Series
policyBellman = pd.Series(index=network.columns, dtype=object)

# Iterate through the nodes to find the optimal policy
for node in policyBellman.index:
    currentNode = node
    
    # If the node is the first node, set the policy to itself
    if node == 'Node 1':
        policyBellman[node] = ('Node 1',)
        continue
    
    # If the node is not the first node, iterate through the predecessors
    # to find the optimal path
    while True:
        # This first statement is necessary to avoid a TypeError
        if type(policyBellman[node]) == float:
            policyBellman[node] = (currentNode,)
        else:
            policyBellman[node] = (currentNode,) + policyBellman[node]
        # Once Node 1 is reached, end the loop
        if currentNode == 'Node 1':
            break
        else:
            currentNode = predecessors[currentNode]

print(policyBellman)

Node 1                                   (Node 1,)
Node 2            (Node 1, Node 3, Node 5, Node 2)
Node 3                            (Node 1, Node 3)
Node 4            (Node 1, Node 3, Node 5, Node 4)
Node 5                    (Node 1, Node 3, Node 5)
Node 6    (Node 1, Node 3, Node 5, Node 4, Node 6)
dtype: object


## Floyd Algorithm

Initialize distance matrix using the original network matrix

In [9]:
distances = network.copy()

display(distances)

,Node 1,Node 2,Node 3,Node 4,Node 5,Node 6
Node 1,0.0,5.0,1.0,NaN,NaN,NaN
Node 2,NaN,0.0,4.0,6.0,NaN,NaN
Node 3,NaN,NaN,0.0,7.0,2.0,NaN
Node 4,NaN,NaN,NaN,0.0,NaN,2.0
Node 5,NaN,1.0,NaN,3.0,0.0,8.0
Node 6,NaN,NaN,NaN,NaN,NaN,0.0


In [10]:
# Iterate through each possible intermediate node
for intermediateNode in distances.index:
    # Iterate through each possible start node
    for startNode in distances.index:
        # Iterate through each possible end node
        for endNode in distances.columns:
            # Update the distance if a shorter path is found
            distances.loc[startNode,endNode] = np.nanmin([distances.loc[startNode,endNode], distances.loc[startNode,intermediateNode] + distances.loc[intermediateNode,endNode]])

display(distances)


/var/folders/v_/lft8__tj6fd561d9kywt6m21d_dv91/T/ipykernel_61990/388568499.py:8: RuntimeWarning: All-NaN axis encountered
  distances.loc[startNode,endNode] = np.nanmin([distances.loc[startNode,endNode], distances.loc[startNode,intermediateNode] + distances.loc[intermediateNode,endNode]])


,Node 1,Node 2,Node 3,Node 4,Node 5,Node 6
Node 1,0.0,4.0,1.0,6.0,3.0,8.0
Node 2,NaN,0.0,4.0,6.0,6.0,8.0
Node 3,NaN,3.0,0.0,5.0,2.0,7.0
Node 4,NaN,NaN,NaN,0.0,NaN,2.0
Node 5,NaN,1.0,5.0,3.0,0.0,5.0
Node 6,NaN,NaN,NaN,NaN,NaN,0.0


### Finding optimal path policy

In [11]:
def findPredecessorsFloyd(startNode, targetNode, adjacencyMatrix, distanceMatrix):
    # Find optimal costs associated with the start node
    optimalCosts = distanceMatrix.loc[startNode].copy()
    
    # Iterate through the nodes that are adjacent to the target node unless it's the start node
    if targetNode == startNode:
        return((startNode,))

    # Find nodes that are adjacent to the target node and set the cost of same node to NaN so
    # that it isn't considered
    connectingCosts = adjacencyMatrix.loc[:,targetNode].copy()
    connectingCosts.loc[targetNode] = np.nan

    # Iterate through adjacent nodes
    for adjacentNode in connectingCosts.index:
        
        # If the node is not adjacent, ignore it
        if pd.isna(connectingCosts[adjacentNode]):
            continue
        
        # Check if the cost to get to an adjacent node + the cost to get to target node is equal or
        # less than the optimal cost--if so, the adjacent node is a predecessor
        if optimalCosts[adjacentNode] + connectingCosts[adjacentNode] == optimalCosts[targetNode]:
            return findPredecessorsFloyd(startNode, adjacentNode, adjacencyMatrix, distanceMatrix) + (targetNode,)

In [12]:
# Save a DataFrame with the optimal policy using the distances matrix
policyFloyd = pd.DataFrame(index=network.index, columns=network.columns, dtype=object)

for startNode in network.index:
    for targetNode in network.columns:
        policyFloyd.loc[startNode, targetNode] = findPredecessorsFloyd(startNode=startNode, targetNode=targetNode, adjacencyMatrix=network, distanceMatrix=distances)

display(policyFloyd)

,Node 1,Node 2,Node 3,Node 4,Node 5,Node 6
Node 1,"(Node 1,)","(Node 1, Node 3, Node 5, Node 2)","(Node 1, Node 3)","(Node 1, Node 3, Node 5, Node 4)","(Node 1, Node 3, Node 5)","(Node 1, Node 3, Node 5, Node 4, Node 6)"
Node 2,None,"(Node 2,)","(Node 2, Node 3)","(Node 2, Node 4)","(Node 2, Node 3, Node 5)","(Node 2, Node 4, Node 6)"
Node 3,None,"(Node 3, Node 5, Node 2)","(Node 3,)","(Node 3, Node 5, Node 4)","(Node 3, Node 5)","(Node 3, Node 5, Node 4, Node 6)"
Node 4,None,None,None,"(Node 4,)",None,"(Node 4, Node 6)"
Node 5,None,"(Node 5, Node 2)","(Node 5, Node 2, Node 3)","(Node 5, Node 4)","(Node 5,)","(Node 5, Node 4, Node 6)"
Node 6,None,None,None,None,None,"(Node 6,)"
